In [ ]:
from inference_sdk import InferenceHTTPClient

CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="2YoFjYTilm3H760rS15g"
)
mypath = "monster_wall.jpeg"
result = CLIENT.infer(mypath, model_id="tin-can-r0yev/1")
#print(result)

from inference import get_model
import supervision as sv
import cv2

# define the image url to use for inference
image_file = "monster_wall.jpeg"
image = cv2.imread(image_file)

# load the results into the supervision Detections api
detections = sv.Detections.from_inference(result)

# create supervision annotators
bounding_box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# annotate the image with our inference results
annotated_image = bounding_box_annotator.annotate(
    scene=image, detections=detections)
annotated_image = label_annotator.annotate(
    scene=annotated_image, detections=detections)

# display the image
#sv.plot_image(annotated_image)

from PIL import Image, ImageDraw
import os

output_folder = "solution1/bounded_images"
# Make sure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Load the original image (use the path to your original image file)
original_image = Image.open(image_file)

# Image dimensions
image_width = result['image']['width']
image_height = result['image']['height']

# Iterate through each prediction and crop the corresponding portion of the image
for i, prediction in enumerate(result['predictions']):
    # Calculate bounding box coordinates
    x = prediction['x']
    y = prediction['y']
    width = prediction['width']
    height = prediction['height']

    # Calculate the top-left and bottom-right corners of the bounding box
    left = x - width / 2
    top = y - height / 2
    right = x + width / 2
    bottom = y + height / 2

    # Make sure the coordinates are within the bounds of the original image
    left = max(0, left)
    top = max(0, top)
    right = min(image_width, right)
    bottom = min(image_height, bottom)

    # Crop the image to the bounding box
    cropped_image = original_image.crop((left, top, right, bottom))

    # Save the cropped image in the specified folder
    output_path = os.path.join(output_folder, f"cropped_image_{i}.jpeg")
    cropped_image.save(output_path)

In [ ]:
import cv2
import numpy as np
import os

# Define input and output directories
input_folder = "solution1/bounded_images"
output_folder = "solution1/enhanced_images"
os.makedirs(output_folder, exist_ok=True)

def enhance_image(image_path, output_path):
    # Load the cropped image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load {image_path}")
        return

    # Step 1: Denoise the image
    denoised = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)

    # Step 2: Apply sharpening filter
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    sharpened = cv2.filter2D(denoised, -1, kernel)

    # Step 3: Upscale the image
    upscaled = cv2.resize(sharpened, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

    # Save the processed image
    cv2.imwrite(output_path, upscaled)
    print(f"Saved enhanced image to {output_path}")

# Process each cropped image
for filename in os.listdir(input_folder):
    input_path = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, f"enhanced_{filename}")
    enhance_image(input_path, output_path)

print("Image enhancement completed.")
